In [124]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from time import sleep

In [125]:
def extrai_dados_preco_popular(ean):
    url_cliquefarma = f'https://www.cliquefarma.com.br/preco/site/farmacia-preco-popular/{ean}'
    headers = {'user-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    site = requests.get(url_cliquefarma, headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')
    link_interno_cliquefarma = soup.find('a', class_='link-oferta')['href']
    site_interno_cliquefarma = requests.get(link_interno_cliquefarma, headers=headers)
    soup_interno_cliquefarma = BeautifulSoup(site_interno_cliquefarma.content, 'html.parser')
    link = soup_interno_cliquefarma.find('a', class_='inline-block')['href']
    link_final_preco_popular = link.split('?idsku')[0]
    site_precopopular = requests.get(link_final_preco_popular, headers=headers)
    soup_site_precopopular = BeautifulSoup(site_precopopular.content, 'html.parser')
    cod_preco_popular = soup_site_precopopular.find('div', class_='productReference').get_text()
    preco_preco_popular = soup_site_precopopular.find('strong', class_='skuBestPrice').get_text()
    preco_preco_popular = float(preco_preco_popular.split('R$')[1].replace(',', '.'))
    lst_preco_popular = [link_final_preco_popular, cod_preco_popular, preco_preco_popular]
    return lst_preco_popular

def extrai_dados_raia(ean):
    url_cliquefarma = f'https://www.cliquefarma.com.br/preco/site/drogaraia/{ean}'
    headers = {'user-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    site = requests.get(url_cliquefarma, headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')
    link_interno_cliquefarma = soup.find('a', class_='link-oferta')['href']
    site_interno_cliquefarma = requests.get(link_interno_cliquefarma, headers=headers)
    soup_interno_cliquefarma = BeautifulSoup(site_interno_cliquefarma.content, 'html.parser')
    link = soup_interno_cliquefarma.find('a', class_='inline-block')['href']
    link_final_raia = f"{'https://drogaraia.com.br/' + link.split('.br%2F')[1].split('%3')[0]}"
    site_raia = requests.get(link_final_raia, headers=headers)
    soup_site_raia = BeautifulSoup(site_raia.content, 'html.parser')
    cod_raia = soup_site_raia.find('span', class_='RaiaProductDescriptionstyles__Data-sc-1ijezxr-8').get_text()
    preco_raia = soup_site_raia.find('span', class_='ProductPricestyles__Price-i0kwh2-4').get_text()
    preco_raia = float(preco_raia.split('R$ ')[1].replace(',', '.'))
    lst_raia = [link_final_raia, cod_raia, preco_raia]
    return lst_raia    

In [126]:
data = pd.read_excel('genericos.xls')
df = data.head(10)
df

,codshopfarma,laboratorio,ean,produto,preco_venda,preco_compra,margem,preco_novo,margem_nova,erro
0,16982,ACHE/BIOSINTETICA,7.896182e+12,ACEBROFILINA XPE AD 120ML,36.50,12.57,190.373906,24.99,98.806683,NaN
1,113267,CIMED,7.896523e+12,ACEBROFILINA XPE AD 120ML,35.00,12.99,169.438029,24.99,92.378753,NaN
2,8731,NEO QUIMICA,7.896714e+12,ACEBROFILINA XPE AD 120ML,32.50,7.99,306.758448,24.99,212.765957,NaN
3,110884,TEUTO,7.896112e+12,ACEBROFILINA XPE AD 120ML,28.00,10.02,88.622754,24.99,149.401198,NaN
4,16545,NOVA QUIMICA,7.895296e+12,ACEBROFILINA XPE AD 120ML,37.05,0.00,inf,24.99,NaN,NaN
5,111516,TEUTO,7.896112e+12,ACEBROFILINA XPE AD 120ML,28.00,0.00,inf,24.99,NaN,NaN
6,102265,EUROFARMA,7.891317e+12,ACEBROFILINA XPE AD 120ML,37.99,8.65,339.190751,24.99,188.901734,NaN
7,2350,GERMED,7.896005e+12,ACEBROFILINA XPE AD 120ML,37.19,6.96,434.339080,24.99,259.051724,NaN
8,104243,EMS,7.896005e+12,ACEBROFILINA XPE AD 120ML,37.99,7.24,424.723757,24.99,245.165746,NaN
9,124425,GEOLAB,7.899095e+12,ACEBROFILINA XPE AD 120ML,32.21,7.94,202.267003,24.99,214.735516,NaN


In [127]:
extrai_dados_preco_popular('7896523206110')

['https://www.precopopular.com.br/cimelide-100mg-com-12-comprimidos/p',
 '695956',
 10.07]

In [128]:
lst_preco_popular = []
for index, row in tqdm(df.iterrows()):
    ean = str(int(row['ean']))
    try:
        product = extrai_dados_preco_popular(ean)
        lst_preco_popular.append(product)
    except:
        pass

lst_preco_popular

10it [00:05,  1.78it/s]


[['https://www.precopopular.com.br/acebrofilina-cimed-120ml-xarope-50mg-5ml-generico/p',
  '741968',
  22.95],
 ['https://www.precopopular.com.br/acebrofilina-eurofarma-50mg-xarope-adulto-120ml/p',
  '697017',
  26.81],
 ['https://www.precopopular.com.br/acebrofilina-ems-50mg-xarope-adulto-120ml/p',
  '701180',
  25.61]]